## Optional Lab: Gradient Descent for Linear Regression

## Goals
In this lab, you will:

automate the process of optimizing $ w $ and $ b $ using gradient descent.

## Tools
In this lab, we will make use of:

NumPy, a popular library for scientific computing
Matplotlib, a popular library for plotting data
plotting routines in the lab_utils.py file in the local directory

In [1]:
import math, copy
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('C:/Users/BM/ML/deeplearning.mplstyle')
from lab_utils_uni import plt_house_x, plt_contour_wgrad, plt_divergence, plt_gradients

## Problem Statement
Let's use the same two data points as before - a house with 1000 square feet sold for $300,000 and a house with 2000 square feet sold for $500,000.

|Size (1000 sqft)	|Price (1000s of dollars) |
|-------------------|-------------------------|
|1               	|300                      |      
|2               	|500                      |  

In [2]:
# load our data set
x_train = np.array([1.0, 2.0]) #features
y_train = np.array([300.0, 500.0])  #target value

## Compute_Cost
This was developed in the last lab. We'll need it again here.


In [3]:

#Function to calculate the cost
def compute_cost(x, y, w, b):
   
    m = x.shape[0] 
    cost = 0
    
    for i in range(m):
        f_wb = w * x[i] + b
        cost = cost + (f_wb - y[i])**2
    total_cost = 1 / (2 * m) * cost

    return total_cost

##  Gradient descent summary
So far in this course, you have developed a linear model that predicts $ f_{w,b}(x^{(i)}) $:
$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1} $$
 
In linear regression, you utilize input training data to fit the parameters $ w,b $ ,  by minimizing a measure of the error between our predictions $ f_{w,b}(x^{(i)}) $  and the actual data $ y^{(i)} $. The measure is called the $ cost, J(w,b) $, . In training you measure the cost over all of our training samples $ x^{(i)},y^{(i)}  $
 
$$ J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2} $$

In lecture, gradient descent was described as:
$$ \begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{3}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*} $$

where, parameters $ w,b $ ,   are updated simultaneously.
The gradient is defined as:
$$ \begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align} $$

Here simultaniously means that you calculate the partial derivatives for all the parameters before updating any of the parameters.

## Implement Gradient Descent
You will implement gradient descent algorithm for one feature. You will need three functions.

* compute_gradient implementing equation (4) and (5) above
* compute_cost implementing equation (2) above (code from previous lab)
* gradient_descent, utilizing compute_gradient and compute_cost
Conventions:

* The naming of python variables containing partial derivatives follows this pattern, $  \frac{\partial J(w,b)}{\partial b} $ will be dj_db.
* w.r.t is With Respect To, as in partial derivative of $  J(wb) $ With Respect To $ b $.

### compute_gradient
compute_gradient implements (4) and (5) above and returns $ \frac{\partial J(w,b)}{\partial w} , \frac{\partial J(w,b)}{\partial b} $. The embedded comments describe the operations.

In [ ]:
def compute_gradient(x, y, w, b): 
    """
    Computes the gradient for linear regression 
    Args:
      x (ndarray (m,)): Data, m examples 
      y (ndarray (m,)): target values
      w,b (scalar)    : model parameters  
    Returns
      dj_dw (scalar): The gradient of the cost w.r.t. the parameters w
      dj_db (scalar): The gradient of the cost w.r.t. the parameter b     
     """
    
    # Number of training examples
    m = x.shape[0]    
    dj_dw = 0
    dj_db = 0
    
    for i in range(m):  
        f_wb = w * x[i] + b 
        dj_dw_i = (f_wb - y[i]) * x[i] 
        dj_db_i = f_wb - y[i] 
        dj_db += dj_db_i
        dj_dw += dj_dw_i 
    dj_dw = dj_dw / m 
    dj_db = dj_db / m 
        
    return dj_dw, dj_db